In [32]:
from nltk.util import pr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from numpy.lib.stride_tricks import as_strided
import matplotlib.pyplot as plt

In [3]:
stopword=set(stopwords.words('english'))
data = pd.read_csv("twitter.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [4]:
data["labels"] = data["class"].map({0: "Hate Speech", 
                                    1: "Offensive Language", 
                                    2: "No Hate and Offensive"})
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet                 labels  
0  !!! RT @mayasolovely: As a woman you shouldn't...  No Hate and Offensive  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...     Offensive Language  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...     Offensive Language  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...     Offensive Language  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     Offensive Language  


In [5]:
data = data[["tweet", "labels"]]
print(data.head())

                                               tweet                 labels
0  !!! RT @mayasolovely: As a woman you shouldn't...  No Hate and Offensive
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...     Offensive Language
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...     Offensive Language
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...     Offensive Language
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     Offensive Language


In [6]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

In [23]:
def binarize(z):
    return (z[:,None]==np.arange(z.max())).astype(int)

In [20]:
data["labels"].replace({"Hate Speech": 0, "Offensive Language": 1, "No Hate and Offensive": 2}, inplace=True)
data

,tweet,labels
0,rt mayasolov woman shouldnt complain clean ho...,2
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,1
2,rt urkindofbrand dawg rt ever fuck bitch sta...,1
3,rt cganderson vivabas look like tranni,1
4,rt shenikarobert shit hear might true might f...,1
...,...,...
24778,yous muthafin lie coreyemanuel right tl tras...,1
24779,youv gone broke wrong heart babi drove redneck...,2
24780,young buck wanna eat dat nigguh like aint fuck...,1
24781,youu got wild bitch tellin lie,1


In [21]:
x = np.array(data["tweet"])
y = np.array(data["labels"])



In [24]:
x = np.array(data["tweet"])
y = np.array(data["labels"])

cv = CountVectorizer()
X = cv.fit_transform(x) # Fit the Data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
y_train = binarize(y_train)
y_test = binarize(y_test)

In [27]:
print("Shape X_train:",x_train.shape)
print("Shape x_test:",x_test.shape)
print("Shape y_train:",y_train.shape)
print("Shape y_test:",y_test.shape)

Shape X_train: (19826, 25693)
Shape x_test: (4957, 25693)
Shape y_train: (19826, 2)
Shape y_test: (4957, 2)


In [28]:
x_train

<19826x25693 sparse matrix of type '<class 'numpy.int64'>'
	with 158277 stored elements in Compressed Sparse Row format>

In [29]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

# Derivative of sigmoid function :
def sigmoid_prime(z):
  return sigmoid(z)*(1-sigmoid(z))

#ReLu
def relu(z):
    return np.maximum(0.01*z,z)

# Derivative for ReLU
def relu_prime(z):
    z[z<=0]=0.01
    z[z>0]=1
    return z

def softmax(z, _axis=0):
    stable_z=z-np.max(z)
    e_z=np.exp(stable_z)
    return e_z/np.sum(e_z,axis=_axis, keepdims=True)

def softmax_prime(z):
    J=-z[...,None]*z[None,...]
    J_diag=np.einsum('ii -> i',J)
    j_diag[:]=z*(1. -z)
    return J.sum(axis=1)

def weight_init(inp,out):
    return np.random.randn(inp,out)/np.sqrt(inp)

In [35]:
def train_batch(units_count, x, y, lr, epochs, bias=False, _seed=42):
    batch_size, ni = x.shape[-2:]
    print(batch_size)
    units_count.insert(0,ni)
    units_count_arr = np.array(units_count)
    L, = units_count_arr.shape # Number of layers + 1
    
    arr_view = as_strided(units_count_arr, shape=(L-1,2), strides=(4,4))
    print(arr_view)
    rng = np.random.default_rng(seed=_seed)
    wghts = [None]*(L-1)
    intercepts = [None]*(L-1)
    # WEIGHTS INITIALIZATION
    for i in range(L-1):
        w_cols, w_rows = arr_view[i,:]
        print("w_cols, w_rows",i,w_rows, w_cols)
        wghts[i] = weight_init(w_rows, w_cols)
        #wghts[i] = rng.random((w_rows, w_cols))
        #print("Weight",i,wghts[i].shape)
        if bias:
            intercepts[i] = rng.random((w_rows,))
            #print("intercepts",i,intercepts[i].shape)
    costs = np.zeros(epochs)
    # Gradient Descent
    for epoch in range(epochs):
        # FORWARD PROPAGATION
        # hidden layer 1 implementation, relu activation
        print(x)
        h1a = np.einsum('hi,Bi -> Bh', wghts[0], x)
        if bias:
            h1a = h1a + intercepts[0]
        h1 = relu(h1a)
        
        
        # hidden layer 2 implementation, sigmoid activation
        h2a = np.einsum('ho,Bo -> Bh', wghts[1], h1) 
        if bias:
            h2a = h2a + intercepts[1]
        h2 = sigmoid(h2a)
        
        # hidden layer 3 implementation, sigmoid activation
        h3a = np.einsum('ho,Bo -> Bh', wghts[2], h2) 
        if bias:
            h3a = h3a + intercepts[2]
        h3 = sigmoid(h3a)
        
        # hidden layer 4 implementation, sigmoid activation
        h4a = np.einsum('ho,Bo -> Bh', wghts[3], h3) 
        if bias:
            h4a = h4a + intercepts[3]
        #hyp = softmax(h4a,_axis=1)
        hyp=sigmoid(h4a)
        
        
        
        current_epoch_cost = -np.einsum('Bi,Bi', y, np.log(hyp))/batch_size
        
        costs[epoch] = current_epoch_cost
        if(epoch%20000==0):
            print("Epoch:",epoch, " Cost:",current_epoch_cost)
    
    
        # BACKWARD PROPAGATION
        # layer 4
        dJ_dH4a = hyp - y
        dJ_dW3 = np.einsum('Bi,Bj -> ij',dJ_dH4a, h3)/batch_size
        
        # layer 3
        dJ_dH3 = np.einsum('Bi,ij -> Bj', dJ_dH4a, wghts[3])
        dJ_dH3a = dJ_dH3*sigmoid_prime(h3a)
        dJ_dW2 = np.einsum('Bi,Bj -> ij',dJ_dH3a, h2)/batch_size
        
        
        # layer 2
        dJ_dH2 = np.einsum('Bi,ij -> Bj', dJ_dH3a, wghts[2])
        dJ_dH2a = dJ_dH2*sigmoid_prime(h2a)
        dJ_dW1 = np.einsum('Bi,Bj -> ij',dJ_dH2a, h1)/batch_size
        
        
        
        # layer 1
        dJ_dH1 = np.einsum('Bi,ij -> Bj', dJ_dH2a, wghts[1])
        dJ_dH1a = dJ_dH1*relu_prime(h1a)
        dJ_dW0 = np.einsum('Bi,Bj -> ij',dJ_dH1a, x)/batch_size
        
        
        if bias:
            dJ_dB3 = np.einsum("Bi -> i", dJ_dH4a)/batch_size
            dJ_dB2 = np.einsum("Bi -> i", dJ_dH3a)/batch_size
            dJ_dB1 = np.einsum("Bi -> i", dJ_dH2a)/batch_size
            dJ_dB0 = np.einsum("Bi -> i",dJ_dH1a)/batch_size
            
        
        # WEIGHTS ADJUSTMENT
        wghts[3] = wghts[3] - lr*dJ_dW3
        wghts[2] = wghts[2] - lr*dJ_dW2
        wghts[1] = wghts[1] - lr*dJ_dW1
        wghts[0] = wghts[0] - lr*dJ_dW0
        if bias:
            intercepts[3] = intercepts[3] - lr*dJ_dB3
            intercepts[2] = intercepts[2] - lr*dJ_dB2
            intercepts[1] = intercepts[1] - lr*dJ_dB1
            intercepts[0] = intercepts[0] - lr*dJ_dB0
    if bias:
        return (costs, wghts, intercepts)
    else:
        return (costs, wghts)

In [36]:
def predict(x,fw,fb):
    h1a = np.einsum('hi,Bi -> Bh', fw[0], x)+fb[0]
    h1 = relu(h1a)
    h2a = np.einsum('hi,Bi-> Bh',fw[1],h1)+fb[1]
    h2=sigmoid(h2a)
    h3a=np.einsum('hi,Bi-> Bh',fw[2],h2)+fb[2]
    h3=sigmoid(h3a)
    h4a=np.einsum('ho,Bo-> Bh',fw[3],h3)+fb[3]
    return sigmoid(h4a)

In [37]:
unit_per_layer_counts = [14,10,14,3]
costs, fw, fb = train_batch(unit_per_layer_counts, x_train, y_train, lr=0.01, epochs=100000, bias=True)
plt.figure(figsize=(40,40))
plt.plot(costs)

19826
[[25693    14]
 [   14    10]
 [   10    14]
 [   14     3]]
w_cols, w_rows 0 14 25693
w_cols, w_rows 1 10 14
w_cols, w_rows 2 14 10
w_cols, w_rows 3 3 14
  (0, 18926)	1
  (0, 23688)	1
  (0, 8329)	1
  (0, 17042)	1
  (0, 17422)	1
  (0, 24595)	1
  (0, 24421)	1
  (0, 16721)	1
  (0, 7700)	1
  (0, 17497)	1
  (0, 6655)	1
  (0, 16592)	1
  (0, 9898)	1
  (0, 8059)	1
  (0, 22045)	1
  (1, 22343)	1
  (1, 19499)	1
  (1, 2094)	1
  (1, 17025)	1
  (1, 8045)	1
  (1, 24986)	1
  (1, 16870)	2
  (1, 11620)	1
  (1, 10793)	1
  (2, 18926)	1
  :	:
  (19822, 19729)	1
  (19822, 25396)	1
  (19822, 2672)	1
  (19822, 6614)	1
  (19823, 17907)	1
  (19823, 9188)	1
  (19823, 17400)	1
  (19823, 17352)	1
  (19823, 664)	2
  (19823, 2839)	1
  (19824, 18926)	1
  (19824, 8329)	1
  (19824, 7392)	1
  (19824, 17788)	1
  (19824, 4668)	1
  (19824, 830)	1
  (19824, 971)	1
  (19824, 16922)	1
  (19824, 22420)	1
  (19824, 11712)	1
  (19824, 3302)	1
  (19824, 1989)	1
  (19825, 2128)	1
  (19825, 19866)	1
  (19825, 19290)	1


ValueError: einstein sum subscripts string contains too many subscripts for operand 1